In [2]:
import os


os.chdir('/Users/seonhwa/Desktop/무제 폴더 3')

print(os.getcwd())  


/Users/seonhwa/Desktop/무제 폴더 3


In [7]:
print("Deployment:", AZURE_OPENAI_DEPLOYMENT)
print("Endpoint:", AZURE_OPENAI_ENDPOINT)


Deployment: gpt-4.1-mini
Endpoint: https://5team-history-azureai-services.openai.azure.com/


In [46]:
import os
from openai import AzureOpenAI
import dotenv

# .env 파일 로드
dotenv.load_dotenv()


AZURE_OPENAI_API_KEY = os.getenv("AZURE_OAI_KEY")
AZURE_OPENAI_ENDPOINT = os.getenv("AZURE_OAI_ENDPOINT")
AZURE_OPENAI_API_VERSION = os.getenv("AZURE_OAI_API_VER")
AZURE_OPENAI_DEPLOYMENT = os.getenv("AZURE_OAI_DEPLOY_NAME")
AZURE_OPENAI_EMBEDDING = os.getenv("AZURE_OAI_EMBEDDING")


# 클라이언트 생성
client = AzureOpenAI(
    api_key= AZURE_OPENAI_API_KEY,
    azure_endpoint= AZURE_OPENAI_ENDPOINT,
    api_version= AZURE_OPENAI_API_VERSION,
)

# 시스템 프롬프트 설정
system_message = (
    "너는 RAG 시스템과 연동된 조선시대 한국사 전문가야. 사용자의 질문에 대해 벡터 검색된 사료 청크를 바탕으로만 답변하고, "
    "반드시 해당 청크에서 발췌한 문장을 명시하거나 출처를 표기해야 해. "
    "사료에 존재하지 않는 정보는 추측하지 말고 '해당 정보는 실록에서 확인되지 않습니다'라고 말해줘."
)

def extract_keywords(query):
    prompt = f"다음 문장에서 핵심 키워드 3~5개만 '키워드1, 키워드2, ...' 형식으로 추출해줘:\n{query}\n>"
    response = client.chat.completions.create(
        model=AZURE_OPENAI_DEPLOYMENT,
        max_tokens=100,
        temperature=0.7,
        messages=[
            {"role": "system", "content": system_message},
            {"role": "user", "content": prompt},
        ],
    )
    keywords = response.choices[0].message.content.strip()
    return [kw.strip() for kw in keywords.split(",")]

def evaluate_answer(keywords, answer):
    eval_prompt = (
        f"다음 키워드: {', '.join(keywords)}\n아래 답변이 키워드를 잘 반영하고 있는지 "
        f"조선시대 한국사 전문가 AI 시각으로 평가해줘:\n{answer}"
    )
    response = client.chat.completions.create(
        model=AZURE_OPENAI_DEPLOYMENT,
        max_tokens=200,
        temperature=0.7,
        messages=[
            {"role": "system", "content": system_message},
            {"role": "user", "content": eval_prompt},
        ],
    )
    return response.choices[0].message.content.strip()

def get_answer(query):
    rag_prompt = (
        f"{query}\n(다음 내용은 조선왕조실록을 참조하여 고증한 내용으로 작성되어야 합니다.)"
    )
    response = client.chat.completions.create(
        model=AZURE_OPENAI_DEPLOYMENT,
        max_tokens=1024,
        temperature=0.7,
        messages=[
            {"role": "system", "content": system_message},
            {"role": "user", "content": rag_prompt},
        ],
    )
    return response.choices[0].message.content.strip()

if __name__ == "__main__":
    query = "조선 중기의 양반들의 삶"

    # 1. 키워드 추출
    keywords = extract_keywords(query)

    # 2. 질문에 대한 답변 생성
    answer = get_answer(query)

    # 3. AI가 답변과 키워드 매칭 평가
    ai_eval = evaluate_answer(keywords, answer)

    # 4. JSON 결과 구성 및 출력
    result_json = {
        "keywords": keywords,
        "ai_eval": ai_eval,
        "answer": answer,
    }
    print(result_json)


{'keywords': ['키워드1: 조선 중기', '키워드2: 양반', '키워드3: 삶'], 'ai_eval': '해당 답변은 조선 중기, 양반, 삶이라는 세 키워드를 전반적으로 잘 반영하고 있습니다. 답변은 조선 중기 양반들이 관직에 나아가 국가에 봉사하거나 향촌에서 향리 역할을 수행했다는 점, 학문과 예절을 중시하며 가문과 자손 교육에 힘썼다는 점, 그리고 지역사회에서 지도자 역할을 하였다는 점을 실록을 인용하여 구체적으로 설명하고 있습니다.\n\n다만, 답변에서 제시한 『선조실록』과 『중종실록』의 구체적인 인용문은 실록 원문에서 발췌된 문장인지 RAG 시스템을 통해 검색된 사료 청크에 실제로 존재하는 문장인지를 명확히 밝혀야 합니다. 조선왕조실록 원문은 매우 방대하고 문어체로 기록되어 있으', 'answer': '조선 중기 양반들의 삶에 대해 조선왕조실록에서 확인된 내용을 바탕으로 답변드립니다.\n\n조선 중기 양반들은 주로 관직에 나아가 국가에 봉사하거나 향촌에서 향리로서의 역할을 수행하였습니다. 실록에는 양반들이 학문과 예절을 중시하며, 가문을 유지하고 사회적 지위를 확립하는 데 힘썼다는 기록이 있습니다. 특히, 양반 가문은 자손 교육에 많은 관심을 기울여 유교 경전을 공부하고 과거 시험을 준비하는 모습이 자주 언급됩니다.\n\n예를 들어, 조선왕조실록 『선조실록』에는 "양반들은 학문에 힘쓰며, 가문을 빛내기 위해 자손 교육에 정성을 다하였다"는 구절이 있습니다. 또한, 『중종실록』에는 "향촌에서 양반은 지역사회의 지도자로서 행정과 치안을 담당하였고, 향약을 통해 공동체 질서를 유지하였다"는 내용이 기록되어 있습니다.\n\n이와 같이 조선 중기 양반들의 삶은 학문과 교육, 관직 활동, 그리고 향촌 사회에서의 지도자 역할이라는 세 축으로 요약할 수 있습니다.\n\n(출처: 조선왕조실록 『선조실록』, 『중종실록』)'}


In [ ]:
import os
import re
from datetime import datetime
from azure.search.documents import SearchClient
from azure.core.credentials import AzureKeyCredential
from openai import AzureOpenAI
import dotenv

# .env 파일 로드
dotenv.load_dotenv()

AZURE_OPENAI_API_KEY = os.getenv("AZURE_OAI_KEY")
AZURE_OPENAI_ENDPOINT = os.getenv("AZURE_OAI_ENDPOINT")
AZURE_OPENAI_API_VERSION = os.getenv("AZURE_OAI_API_VER")
AZURE_OPENAI_DEPLOYMENT = os.getenv("AZURE_OAI_DEPLOY_NAME")
AZURE_OPENAI_EMBEDDING = os.getenv("AZURE_OAI_EMBEDDING")


def extract_keywords(query):
    prompt = f"""다음 문장에서 한국사 문서 검색을 위한 핵심 키워드 5개만 콤마로 출력:
{query}
키워드:"""
    response = openai_client.chat.completions.create(
        model=AZURE_OPENAI_DEPLOYMENT,
        max_tokens=50,
        temperature=0.2,
        messages=[
            {"role": "system", "content": "키워드만 콤마로 출력"},
            {"role": "user", "content": prompt}
        ]
    )
    kw_line = response.choices[0].message.content.strip()
    return [kw.strip() for kw in re.split(r'[,\n]', kw_line) if len(kw.strip()) > 0]

def search_documents(keywords, top_k=5):
    docs = []
    for kw in keywords:
        try:
            results = search_client.search(kw, top=top_k, search_mode="any")
            for doc in results:
                # "chunk" 필드 등 실제 문서 내용 필드 명칭에 맞게 수정
                chunk = doc.get('chunk', '')
                ref = doc.get('title', '')  # 출처/제목/연-월-일 등
                if chunk and len(chunk) > 50:
                    docs.append({"keyword": kw, "content": chunk, "ref": ref})
        except Exception:
            continue
    # 중복 제거(내용 기준)
    seen = set()
    unique_docs = []
    for d in docs:
        key = d['content'][:60]
        if key not in seen:
            seen.add(key)
            unique_docs.append(d)
    return unique_docs[:3]

def create_synopsis(query, docs):
    context = "\n".join([d['content'] for d in docs])
    prompt = f"""질문: {query}
아래 사료를 참고하여 500자 이내로 드라마틱한 시놉시스를 작성하고, 참조한 사료의 출처문장도 같이 알려줘.
위 자료를 바탕으로 웹툰/드라마 스타일의 흥미진진한 시놉시스를 작성해주세요!

스타일 요구사항:
1. 400-500자 분량
2. 웹툰이나 사극 드라마처럼 흥미진진하고 역동적으로 작성
3. "~다!", "~었다!", "하지만!", "그런데!" 같은 감탄부호 적극 활용
4. 인물들의 감정과 갈등을 생생하게 묘사
5. 긴장감과 반전이 있는 스토리텔링
6. 독자가 "다음이 궁금해!"라고 느낄 수 있게 작성
7. 구체적인 인물명과 사건을 포함하되 드라마틱하게 각색

예시 톤:
"왕좌를 둘러싼 치열한 권력투쟁이 시작된다! 영조는 외쳤다 - '이 나라에 붕당은 필요 없다!' 하지만 신하들의 속마음은 달랐다..."

시놉시스:
사료:
{context}
시놉시스 및 출처:"""
    response = openai_client.chat.completions.create(
        model=AZURE_OPENAI_DEPLOYMENT,
        max_tokens=1000,
        temperature=0.8,
        messages=[{"role": "system", "content": "짧고 생생하게 요약, 출처도 명확히."},
                  {"role": "user", "content": prompt}]
    )
    text = response.choices[0].message.content.strip()
    # 출처 분리 시도
    if "출처:" in text:
        synopsis, refs = text.split("출처:", 1)
    else:
        synopsis, refs = text, ""
    return synopsis.strip(), refs.strip()

def factual_similarity(synopsis, ground_truth):
    prompt = f"""학생이 쓴 요약: {synopsis}
수능특강 설명 원문: {ground_truth}
두 내용의 사실적 일치도(팩트, 시대/주제 부합 등)를 5점 만점으로 수치와 설명으로 평가해줘."""

    response = openai_client.chat.completions.create(
        model=AZURE_OPENAI_DEPLOYMENT,
        max_tokens=500,
        temperature=0.7,
        messages=[{"role": "system", "content": "사실성 평가 전문가"},
                  {"role": "user", "content": prompt}]
    )
    return response.choices[0].message.content.strip()

if __name__ == "__main__":
    query = "조선시대 왕과 신하 갈등"
    ground_truth = "실제 '수능특강' 교재 해당 단원 요약문 전체를 붙이면 됩니다."
    # 1. 키워드
    keywords = extract_keywords(query)
    # 2. 사용 사료/출처 확보
    docs = search_documents(keywords, top_k=5)
    # 3. 시놉시스+출처 생성
    synopsis, refs = create_synopsis(query, docs)
    # 4. 팩트기반 유사성 평가
    factual_eval = factual_similarity(synopsis, ground_truth)

    print("\n" + "=" * 50)
    print("❶ [질문에서 뽑아낸 키워드]")
    print(keywords)
    print("\n❷ [답변 시놉시스]")
    print(synopsis)
    print("\n❸ [답변에 활용한 사료 출처]")
    if refs:
        print(refs)
    else:
        for d in docs:
            print(f"- [{d['ref']}] : {d['content'][:70]}...")
    print("\n❹ [수능특강 비교 - 사실/맥락 평가]")
    print(factual_eval)
    
    #쿼리로 



❶ [질문에서 뽑아낸 키워드]
['조선시대', '왕', '신하', '갈등', '한국사']

❷ [답변 시놉시스]
조선 말, 왕권은 점점 약해지고 신하들의 권력 다툼은 격렬해졌다! 순조가 궁궐 깊숙이 숨자, 김조순 일가가 국정을 장악, 권세를 휘둘렀다! “내가 조선을 좌지우지한다!” 김조순의 야망은 끝이 없었다. 그러나 이들의 세도 정치에 반발하는 목소리가 점점 커졌다. 탕평책을 시도했던 전대의 왕들은 신하들의 당파 싸움과 음모에 지쳐갔다. 신하들은 겉으론 충성을 맹세했지만, 속으로는 권력 쟁탈전의 칼날을 겨누었다! 한편, 문과 무과 잡과로 나뉜 관리 등용 시험 ‘과거’는 왕과 신하 간 신뢰를 시험하는 무대가 되었으니, 왕의 ‘책문’에 응답하는 신하들의 대책 속에 감춰진 속내가 폭발한다! 하지만 세도 정치의 그늘 아래, 조선의 미래는 불투명하기만 하다! 다음 이야기는 과연 권력을 쥔 자들의 운명과 나라의 운명이 어떻게 얽히게 될지, 궁금하지 않은가?

❸ [답변에 활용한 사료 출처]
황현, 『오하기문』(25013-0044), 과거 제도와 탕평 정치 관련 자료(25013-0041, 25013-0043, 25013-0042)

❹ [수능특강 비교 - 사실/맥락 평가]
사실적 일치도 점수: 4.5/5

설명: 학생의 요약은 조선 말기의 정치 상황, 즉 왕권 약화와 신하들의 권력 다툼, 세도 정치의 형성, 그리고 김조순 일가의 국정 장악에 대한 내용을 충실히 반영하고 있습니다. 또한 탕평책과 과거 시험이 정치적 갈등과 신뢰 문제의 무대로 작용했다는 점도 적절히 언급되어 있어 시대적 배경과 주제가 잘 부합합니다. 다만, 일부 표현이 다소 문학적 과장이나 해석적 요소를 포함하고 있어, 원문의 객관적 사실 서술과 완전히 일치한다고 보기는 어렵지만, 핵심적인 역사적 사실과 흐름은 정확히 담고 있습니다. 따라서 전체적인 사실성은 매우 높으나, 원문과 완전 동일한 사실 전달이라는 점에서 약간 감점하였습니다.
